# Letterboxd Profile Analysis

As any avid movie-watcher knows, keeping track of the movies one watches provides one with a personalized cinematic journal. This practice can offer a sense of satisfaction, as well as allows you to reflect on your evolving interests and tastes over time. There are several websites that allow you to do this, such as letterboxd, IMDb, Criticker, and Reelgood. These websites are perfectly fine for keeping track of your movies, and also provide a large community of likeminded movie-watchers, they don't provide much insight to anything else.

In this project, I'd like to focus on some of the quantifiable aspects of all the movies I've watched, as well as some of my movie watching habits. Letterboxd allows its users to download a folder with rudimentary information, such as movies watched, and when they were watched. I will use this csv, along with the TMDb API, to scrape some more relevant, useful data.

Some specific questions I'd like answered are:
- Who are my favorite actors/directors?
- How much time have I spent watching movies?
- How often do I watch movies?
- What type of movies do I tend to gravitate towards?


## 1- First, lets read in and prepare our datasets.

In [1]:
import pandas as pd

df1 = pd.read_csv("/Users/erickpuntiel/Desktop/watched.csv")
df2 = pd.read_csv("/Users/erickpuntiel/Desktop/ratings.csv")

## getting rid of redundant columns
df1 = df1.drop(columns = ['Letterboxd URI'])
df2 = df2.drop(columns = ['Letterboxd URI', 'Date', 'Year'])

## merging dataframes on the movie title
movies = df1.merge(df2, on = 'Name', how = 'left')
movies

,Date,Name,Year,Rating
0,2021-01-16,La Haine,1995,5.0
1,2022-01-07,Oldboy,2003,3.0
2,2022-01-07,No Country for Old Men,2007,5.0
3,2022-01-07,Taxi Driver,1976,NaN
4,2022-01-10,Lady Bird,2017,4.0
...,...,...,...,...
411,2023-08-16,No Hard Feelings,2023,2.0
412,2023-08-24,Knight and Day,2010,3.0
413,2023-08-24,Fatal Attraction,1987,2.0
414,2023-08-25,Point Break,1991,4.0


This is pretty much all the relevant data that the letterboxd csv provides. I'm going to use the tmdb API to scrape more data, such as release year, genre, country of origin, etc.
## 2- Connect to the API, and search for IDs

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = '022c2850fb876773a9522f007ef2c46d'

search = tmdb.Search()
discover = tmdb.Discover()


tmdbID = []

## search by title and year to avoid any redundancies (movies with same title)
for t, y in zip(movies["Name"], movies["Year"]):
    movie_info = search.movie(query = t, year = y)
    if movie_info['results']:
        tmdbID.append(movie_info['results'][0]['id'])
    else:
        tmdbID.append(-1)

print(tmdbID[:15])

[406, 670, 6977, 103, 391713, 194662, 598, 141, 153, 244786, 27205, 38, 1359, 9100, 9424]


## 3- Use each movie's ID to get more information

In [3]:
## runtime, language, country, genre
language, runtime, genre, country = [], [], [], []

for ID in tmdbID:
    if ID != -1:
        movie_info = tmdb.Movies(ID).info()
        language.append(movie_info.get('original_language'))
        runtime.append(movie_info.get('runtime'))
        genre.append(movie_info.get('genres')[0]['name'])
        if movie_info.get("production_countries"):
            country.append(movie_info.get('production_countries')[0]['name'])
        else:
            country.append(None)
    else:
        language.append(None)
        runtime.append(None)
        genre.append(None)
        country.append(None)


In [4]:
movies['TMDB ID'] = tmdbID       
movies['Language'] = language
movies['Runtime'] = runtime
movies['Genre'] = genre
movies['Country'] = country

In [5]:
movies.head(15)

,Date,Name,Year,Rating,TMDB ID,Language,Runtime,Genre,Country
0,2021-01-16,La Haine,1995,5.0,406,fr,98.0,Drama,France
1,2022-01-07,Oldboy,2003,3.0,670,ko,120.0,Drama,South Korea
2,2022-01-07,No Country for Old Men,2007,5.0,6977,en,122.0,Crime,United States of America
3,2022-01-07,Taxi Driver,1976,NaN,103,en,114.0,Crime,United States of America
4,2022-01-10,Lady Bird,2017,4.0,391713,en,94.0,Drama,United States of America
5,2022-01-13,Birdman or (The Unexpected Virtue of Ignorance),2014,3.0,194662,en,120.0,Drama,United States of America
6,2022-01-14,City of God,2002,4.0,598,pt,130.0,Drama,Brazil
7,2022-01-15,Donnie Darko,2001,2.0,141,en,114.0,Fantasy,United States of America
8,2022-01-16,Lost in Translation,2003,5.0,153,en,102.0,Drama,Japan
9,2022-01-17,Whiplash,2014,4.0,244786,en,107.0,Drama,United States of America


We have more information to work with, but I'd like to know less about the movie itself, and moreso the people who worked on the movies. Unlike movie information, crew information is stored in .credits(), not in .info().


In [6]:
directors = []

for id in tmdbID:
    if id == -1:
        directors.append(["No director found"])
        continue  

    credits = tmdb.Movies(id).credits()
    if credits and 'crew' in credits:
        director_names = []
        for crew in credits['crew']:
            if crew['job'] == 'Director':
                director_names.append(crew['name'])
        directors.append(director_names)
        
for director in directors[:15]:
    print(f"Director(s): {', '.join(director)}")

Director(s): Mathieu Kassovitz
Director(s): Park Chan-wook
Director(s): Joel Coen, Ethan Coen
Director(s): Martin Scorsese
Director(s): Greta Gerwig
Director(s): Alejandro González Iñárritu
Director(s): Fernando Meirelles
Director(s): Richard Kelly
Director(s): Sofia Coppola
Director(s): Damien Chazelle
Director(s): Christopher Nolan
Director(s): Michel Gondry
Director(s): Mary Harron
Director(s): Andrew Fleming
Director(s): David Nutter


Since actor information is also stored in .credits(), we can use a similar method to find all the actors. Because there are typically more actors than directors working on a movie, I'm going to find the top 8 billed actors rather than just the leading actor. While this might not accurately account for movies with large ensemble casts, most movies rarely have more than 5 protagonists/antagonists. 

In [7]:
actors = []

for id in tmdbID:
    if id == -1:
        actors.append(["No actors found"])
        continue  

    credits = tmdb.Movies(id).credits()
    if credits and 'cast' in credits:
        actor_names = []
        for cast_member in credits['cast']:
            actor_names.append(cast_member['name'])
            if len(actor_names) == 8:  # Stop after finding the top 8 actors
                break
        actors.append(actor_names)

for actor_names in actors[:15]:
    print(f"Top Actors: {', '.join(actor_names)}")

Top Actors: Vincent Cassel, Hubert Koundé, Saïd Taghmaoui, Abdel Ahmed Ghili, Solo, Joseph Momo, Héloïse Rauth, Rywka Wajsbrot
Top Actors: Choi Min-sik, Yoo Ji-tae, Kang Hye-jung, Kim Byeong-ok, Ji Dae-han, Oh Dal-su, Lee Seung-shin, Yoon Jin-seo
Top Actors: Tommy Lee Jones, Javier Bardem, Josh Brolin, Woody Harrelson, Kelly Macdonald, Garret Dillahunt, Tess Harper, Barry Corbin
Top Actors: Robert De Niro, Jodie Foster, Cybill Shepherd, Harvey Keitel, Peter Boyle, Leonard Harris, Albert Brooks, Diahnne Abbott
Top Actors: Saoirse Ronan, Laurie Metcalf, Tracy Letts, Beanie Feldstein, Lucas Hedges, Timothée Chalamet, Stephen McKinley Henderson, Jordan Rodrigues
Top Actors: Michael Keaton, Emma Stone, Zach Galifianakis, Edward Norton, Andrea Riseborough, Amy Ryan, Naomi Watts, Lindsay Duncan
Top Actors: Alexandre Rodrigues, Leandro Firmino, Phellipe Haagensen, Douglas Silva, Jonathan Haagensen, Matheus Nachtergaele, Seu Jorge, Jefechander Suplino
Top Actors: Jake Gyllenhaal, Jena Malone, J

In [8]:
movies['Director'] = directors
movies['Top 8 Actors'] = actors
movies.head(15)


,Date,Name,Year,Rating,TMDB ID,Language,Runtime,Genre,Country,Director,Top 8 Actors
0,2021-01-16,La Haine,1995,5.0,406,fr,98.0,Drama,France,[Mathieu Kassovitz],"[Vincent Cassel, Hubert Koundé, Saïd Taghmaoui..."
1,2022-01-07,Oldboy,2003,3.0,670,ko,120.0,Drama,South Korea,[Park Chan-wook],"[Choi Min-sik, Yoo Ji-tae, Kang Hye-jung, Kim ..."
2,2022-01-07,No Country for Old Men,2007,5.0,6977,en,122.0,Crime,United States of America,"[Joel Coen, Ethan Coen]","[Tommy Lee Jones, Javier Bardem, Josh Brolin, ..."
3,2022-01-07,Taxi Driver,1976,NaN,103,en,114.0,Crime,United States of America,[Martin Scorsese],"[Robert De Niro, Jodie Foster, Cybill Shepherd..."
4,2022-01-10,Lady Bird,2017,4.0,391713,en,94.0,Drama,United States of America,[Greta Gerwig],"[Saoirse Ronan, Laurie Metcalf, Tracy Letts, B..."
5,2022-01-13,Birdman or (The Unexpected Virtue of Ignorance),2014,3.0,194662,en,120.0,Drama,United States of America,[Alejandro González Iñárritu],"[Michael Keaton, Emma Stone, Zach Galifianakis..."
6,2022-01-14,City of God,2002,4.0,598,pt,130.0,Drama,Brazil,[Fernando Meirelles],"[Alexandre Rodrigues, Leandro Firmino, Phellip..."
7,2022-01-15,Donnie Darko,2001,2.0,141,en,114.0,Fantasy,United States of America,[Richard Kelly],"[Jake Gyllenhaal, Jena Malone, James Duval, Dr..."
8,2022-01-16,Lost in Translation,2003,5.0,153,en,102.0,Drama,Japan,[Sofia Coppola],"[Bill Murray, Scarlett Johansson, Giovanni Rib..."
9,2022-01-17,Whiplash,2014,4.0,244786,en,107.0,Drama,United States of America,[Damien Chazelle],"[Miles Teller, J.K. Simmons, Paul Reiser, Meli..."


Now that we have more workable information, lets do some preliminary cleaning before answering our questions and heading to Tableau. 
- Manually fill in any movies who's data could not be retrieved using TMDB
- Rather than keeping the abbreviation for the langauge, I want the full name of the language. 
- Since runtimes are all rounded, there's no point in keeping the decimal.
- Although some directors share directing credits, I only want to keep the top-billed director.

## 4- Do some preliminary cleaning

In [9]:
## all movies whose data could not be found
print(movies[movies['TMDB ID'] == -1])

           Date                 Name  Year  Rating  TMDB ID Language  Runtime  \
377  2023-06-09  Jackals & Fireflies  2023     NaN       -1     None      NaN   

    Genre Country             Director       Top 8 Actors  
377  None    None  [No director found]  [No actors found]  


In [10]:
# manually filling in data
new_values = {'Director' : 'Charlie Kaufman', 'Top 8 Actors' : ['Eva H.D'],
'Language' : 'en', 'Runtime' : 20.0, 'Genre' : 'Drama',
  'Country' : 'United States of America'}

movies.loc[377, ['Director', 'Top 8 Actors', 'Language', 'Runtime', 'Genre', 'Country']] = new_values
print(movies[movies['TMDB ID'] == -1])

           Date                 Name  Year  Rating  TMDB ID Language  Runtime  \
377  2023-06-09  Jackals & Fireflies  2023     NaN       -1       en     20.0   

     Genre                   Country         Director Top 8 Actors  
377  Drama  United States of America  Charlie Kaufman    [Eva H.D]  


In [11]:
print(movies['Language'].unique())
# dictionary of abbreviation : full name pairs
language_mapping = {
    'fr' : 'French', 'ko' : 'Korea',
    'en' : 'English', 'pt' : 'Portuguese',
    'ja' : 'Japanese', 'ru' : 'Russian',
    'cn' : 'Chinese', 'es' : 'Spanish',
    'sv' : 'Swedish', 'zh' : 'Taiwanese',
    'de' : 'German', 'el' : 'Greek',
    'fi' : 'Finnish', 'fa' : 'Persian'
}

# replace abbreviations with full names using the dictionary
movies['Language'] = movies['Language'].replace(language_mapping)

['fr' 'ko' 'en' 'pt' 'ja' 'ru' 'cn' 'es' 'sv' 'zh' 'de' 'el' 'fi' 'fa']


In [12]:
# dropping the decimal from the runtimes
movies['Runtime'] = movies['Runtime'].astype(int)

In [13]:
# only returning the top billed director
def top_billed(lst):
    if len(lst) > 0:
        return lst[0]
    
movies['Director'] = movies['Director'].apply(top_billed)
movies.head(15)

,Date,Name,Year,Rating,TMDB ID,Language,Runtime,Genre,Country,Director,Top 8 Actors
0,2021-01-16,La Haine,1995,5.0,406,French,98,Drama,France,Mathieu Kassovitz,"[Vincent Cassel, Hubert Koundé, Saïd Taghmaoui..."
1,2022-01-07,Oldboy,2003,3.0,670,Korea,120,Drama,South Korea,Park Chan-wook,"[Choi Min-sik, Yoo Ji-tae, Kang Hye-jung, Kim ..."
2,2022-01-07,No Country for Old Men,2007,5.0,6977,English,122,Crime,United States of America,Joel Coen,"[Tommy Lee Jones, Javier Bardem, Josh Brolin, ..."
3,2022-01-07,Taxi Driver,1976,NaN,103,English,114,Crime,United States of America,Martin Scorsese,"[Robert De Niro, Jodie Foster, Cybill Shepherd..."
4,2022-01-10,Lady Bird,2017,4.0,391713,English,94,Drama,United States of America,Greta Gerwig,"[Saoirse Ronan, Laurie Metcalf, Tracy Letts, B..."
5,2022-01-13,Birdman or (The Unexpected Virtue of Ignorance),2014,3.0,194662,English,120,Drama,United States of America,Alejandro González Iñárritu,"[Michael Keaton, Emma Stone, Zach Galifianakis..."
6,2022-01-14,City of God,2002,4.0,598,Portuguese,130,Drama,Brazil,Fernando Meirelles,"[Alexandre Rodrigues, Leandro Firmino, Phellip..."
7,2022-01-15,Donnie Darko,2001,2.0,141,English,114,Fantasy,United States of America,Richard Kelly,"[Jake Gyllenhaal, Jena Malone, James Duval, Dr..."
8,2022-01-16,Lost in Translation,2003,5.0,153,English,102,Drama,Japan,Sofia Coppola,"[Bill Murray, Scarlett Johansson, Giovanni Rib..."
9,2022-01-17,Whiplash,2014,4.0,244786,English,107,Drama,United States of America,Damien Chazelle,"[Miles Teller, J.K. Simmons, Paul Reiser, Meli..."


## 5- Answer some questions.

Who are my favorite actors/directors? I'm going to do this by counting how many movies by each director/actor, as well as get the average ratings of each actor/director.

In [14]:
directors_count = movies['Director'].value_counts()
actors_count = movies.explode('Top 8 Actors')['Top 8 Actors'].value_counts()

In [15]:
## favorite actors/directors by avg rating
## to be more accurate, we're going to filter out actors/directors who i've seen less than 3 movies

## group by
director_group = movies.groupby('Director')
actor_group = movies.explode('Top 8 Actors').groupby('Top 8 Actors')

## count the number of movies
director_count = director_group['Name'].count()
actor_count = actor_group['Name'].count()

## filter out with 3< movies
atleast_3_dir = director_count[director_count >= 3].index
atleast_3_act = actor_count[actor_count >= 3].index

## calculate the average rating for each director
director_average_rating = director_group['Rating'].mean()
actor_average_rating = actor_group['Rating'].mean()

## sort by average rating
sorted_directors = director_average_rating[atleast_3_dir].sort_values(ascending = False)
sorted_actors = actor_average_rating[atleast_3_act].sort_values(ascending=False)

In [16]:
print('Most Watched Directors')
print(directors_count.head(15))
print('')
print(sorted_directors.head(15))

Most Watched Directors
Martin Scorsese         13
David Cronenberg        10
Wes Anderson             9
Joel Coen                9
Brian De Palma           8
Richard Linklater        8
Quentin Tarantino        7
David Fincher            7
Paul Thomas Anderson     7
Denis Villeneuve         6
Jim Jarmusch             5
Steven Spielberg         5
Christopher Nolan        5
Spike Lee                5
Sidney Lumet             4
Name: Director, dtype: int64

Director
Paul Schrader           4.500000
Spike Lee               4.250000
Stanley Kubrick         4.000000
Richard Linklater       4.000000
Alfonso Cuarón          4.000000
Martin McDonagh         3.666667
Martin Scorsese         3.555556
Quentin Tarantino       3.500000
Sidney Lumet            3.500000
Roman Polanski          3.500000
Ridley Scott            3.500000
Kathryn Bigelow         3.500000
David Fincher           3.500000
Paul Thomas Anderson    3.428571
Jim Jarmusch            3.400000
Name: Rating, dtype: float64


My most watched directors are Martin Scorcese, David Cronenberg, and the Coen brothers/Wes Anderson. They have respective average ratings of 3.55/5, 2.0/5, 3.33/5, and 3.0/5.

With a total of 8 movies watched and an average rating of 4.0/5.0, Richard Linklater is my favorite director.
Although Paul Schrader and Spike Lee have higher average ratings of 4.5 and 4.25, I've only seen 3 and 5 movies by them respectively. 

In [17]:
print('Most Watched Actors')
print(actors_count.head(20))
print('')

Most Watched Actors
Brad Pitt                 16
Robert De Niro            11
Willem Dafoe              10
Frances McDormand         10
Al Pacino                  9
Philip Seymour Hoffman     9
Leonardo DiCaprio          9
Steve Buscemi              7
Woody Harrelson            7
Tom Cruise                 7
Laura Dern                 7
Edward Norton              7
Bill Murray                7
Morgan Freeman             7
Tilda Swinton              7
Keanu Reeves               6
Harvey Keitel              6
Jack Nicholson             6
Sean Penn                  6
Scarlett Johansson         6
Name: Top 8 Actors, dtype: int64



In [18]:
print(sorted_actors.head(15))

Top 8 Actors
Vincent Cassel           5.000000
Spike Lee                5.000000
Justin Theroux           4.666667
Robert Forster           4.500000
Anna Paquin              4.500000
Zoë Kravitz              4.000000
Joe Pesci                4.000000
Daryl Hannah             4.000000
Diane Keaton             4.000000
Dustin Hoffman           4.000000
Gael García Bernal       4.000000
Harry Dean Stanton       4.000000
Julie Delpy              4.000000
Christopher Lloyd        4.000000
Arnold Schwarzenegger    4.000000
Name: Rating, dtype: float64


My most watched actors are Brad Pitt, Robert De Niro and Frances McDormand/Willem Dafoe. They have respective ratings of 3.08/5, 3.375/5, 3.09/5, and a 3.0/5.

With a 5.0/5.0 rating and 3 movies each, Spike Lee and Vincent Cassel are my highest rated actors. With 6 movies watched, and a 4.67/5.0, Justin Theroux is my favorite actor. Naomi Watts is also fairly high up, with 6 movies watched and a 4.0/5.0.

In [22]:
## How much time have I spent watching movies?
total_watched = movies['Runtime'].sum()
print(total_watched, 'minutes.')
print(f"{total_watched / 60.0:.2f} hours.")

print()
start = pd.to_datetime('2022-01-07')
end = pd.to_datetime('2023-08-26')

time = end - start  ## total days i've logged my movies
avg = movies['Runtime'].mean()  ## average length of a movie i watch
total = len(movies['Name']) ## total number of movies i've watched

print('I watch %.2f movies per day, for a total of %.2f minutes a day\n' % (total/596, (total/596)*avg))

days = movies['Date'].dt.day_name().value_counts()
print(days)


48687 minutes.
811.45 hours.

I watch 0.70 movies per day, for a total of 81.69 minutes a day



AttributeError: Can only use .dt accessor with datetimelike values

In [23]:
## what genre of movies do i gravitate towards?
genre_count = movies['Genre'].value_counts()
country_count = movies['Country'].value_counts()

print(genre_count.head(15))

genre_avg = movies.groupby('Genre')['Rating'].mean()
country_avg = movies.groupby('Country')['Rating'].mean()

Drama              131
Comedy              75
Crime               46
Action              45
Horror              38
Thriller            20
Adventure           16
Science Fiction     12
Mystery             11
Romance              7
Fantasy              5
Documentary          4
Animation            3
Family               2
War                  1
Name: Genre, dtype: int64


In [24]:
print(genre_avg.sort_values(ascending = False))

Genre
Family             4.000000
Drama              3.317308
Crime              3.243243
Comedy             3.209677
Science Fiction    3.181818
Adventure          3.090909
Animation          3.000000
Documentary        3.000000
Romance            2.800000
Action             2.787879
Mystery            2.555556
Thriller           2.538462
Fantasy            2.400000
Horror             2.264706
War                     NaN
Name: Rating, dtype: float64


With the exception of the family genre, of which I've only seen 2 movies, my 3 most watched genres are also my 3 highest rated genres. It seems like horror, with a total of 37 movies watched and only a 2.24 rating is my least-favorite genre. 

In [25]:
print(country_count.head(15))

United States of America    234
United Kingdom               33
France                       32
Canada                       23
Germany                      15
Japan                        15
South Korea                   7
Australia                     7
Spain                         6
Mexico                        5
Hong Kong                     4
China                         4
Austria                       3
Soviet Union                  3
Brazil                        3
Name: Country, dtype: int64


In [26]:
print(country_avg.sort_values(ascending = False).head(15))

Country
Turkey                      5.000000
Ireland                     4.000000
Czech Republic              4.000000
Austria                     4.000000
Finland                     4.000000
Uruguay                     4.000000
Japan                       3.727273
Brazil                      3.666667
Mexico                      3.600000
Soviet Union                3.500000
France                      3.440000
Australia                   3.428571
South Korea                 3.333333
United States of America    3.033333
Germany                     3.000000
Name: Rating, dtype: float64


Unsurprisingly, the United States of America is my most watched, as Hollywood dominates the film industry. With 15 movies from Japan, at an average rating of 3.72, Japan makes my favorite movies. France, with 33 movies watched and an average rating of 3.44 is also fairly high up. 

## 6- Head to Tableau.

In [27]:
## now that we've answered some questions, we can head to tableau to start visualizing.
movies.to_csv('letterboxd.csv') 